# Writer 工具

本 Notebook 快速概述了如何开始使用 Writer 的 [tools](https://python.langchain.com/docs/concepts/tools/)。如需了解所有 Writer 功能和配置的详细文档，请前往 [Writer 文档](https://dev.writer.com/home)。

## 概述

### 集成详情

| 类                                                                                                         | 包裹           | 本地 | 可序列化 | JS 支持 |                                                                    包裹下载                                                                    |                                                                    最新包裹                                                                    |
|:-----------------------------------------------------------------------------------------------------------|:---------------|:----:|:-------:|:-------:|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------:|
| [GraphTool](https://github.com/writer/langchain-writer/blob/main/langchain_writer/tools.py#L9) | [langchain-writer](https://pypi.org/project/langchain-writer/) |  ❌   |    ❌    |   ❌    | ![PyPI - Downloads](https://img.shields.io/pypi/dm/langchain-writer?style=flat-square&label=%20) | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-writer?style=flat-square&label=%20) |

### 功能

我们提供了两种工具供 `ChatWriter` 使用：`function` 和 `graph`。

#### Function

函数是最常见的工具类型，它允许 LLM 调用外部 API、从数据库获取数据以及执行任何您想要执行的外部操作。请访问我们的 [工具调用文档](https://dev.writer.com/api-guides/tool-calling#tool-calling) 获取更多信息。

#### Graph

`Graph` 工具是 Writer 的基于图的检索增强生成 (RAG)，称为知识图谱。此工具使开发人员能够仅将图谱 ID 传递给模型，模型将返回提示中问题的答案。要了解更多信息，请参阅我们的 [知识图谱 API 文档](https://dev.writer.com/api-guides/knowledge-graph)。

## 设置

注册 [Writer AI Studio](https://app.writer.com/aistudio/signup?utm_campaign=devrel) 以生成 API 密钥（您可以遵循此 [快速入门指南](https://dev.writer.com/api-guides/quickstart)）。然后，设置 WRITER_API_KEY 环境变量：

In [ ]:
import getpass
import os

if not os.getenv("WRITER_API_KEY"):
    os.environ["WRITER_API_KEY"] = getpass.getpass("Enter your Writer API key: ")

## 用法

您可以将图表或函数工具绑定到 `ChatWriter`。

### 图工具

要绑定图工具，首先使用您想用作源的 `graph_ids` 创建并初始化一个 `GraphTool` 实例：

In [ ]:
from langchain_writer.chat_models import ChatWriter
from langchain_writer.tools import GraphTool

chat = ChatWriter()

graph_id = getpass.getpass("Enter Writer Knowledge Graph ID: ")
graph_tool = GraphTool(graph_ids=[graph_id])

## 实例化

In [ ]:
from typing import Optional

from langchain_core.tools import tool
from pydantic import BaseModel, Field


@tool
def get_supercopa_trophies_count(club_name: str) -> Optional[int]:
    """Returns information about supercopa trophies count.

    Args:
        club_name: Club you want to investigate info of supercopa trophies about

    Returns:
        Number of supercopa trophies or None if there is no info about requested club
    """

    if club_name == "Barcelona":
        return 15
    elif club_name == "Real Madrid":
        return 13
    elif club_name == "Atletico Madrid":
        return 2
    else:
        return None


class GetWeather(BaseModel):
    """Get the current weather in a given location"""

    location: str = Field(..., description="The city and state, e.g. San Francisco, CA")


get_product_info = {
    "type": "function",
    "function": {
        "name": "get_product_info",
        "description": "Get information about a product by its id",
        "parameters": {
            "type": "object",
            "properties": {
                "product_id": {
                    "type": "number",
                    "description": "The unique identifier of the product to retrieve information for",
                }
            },
            "required": ["product_id"],
        },
    },
}

### 绑定工具
然后，你可以简单地将所有工具绑定到 `ChatWriter` 实例：

In [ ]:
chat.bind_tools(
    [graph_tool, get_supercopa_trophies_count, GetWeather, get_product_info]
)

所有工具都存储在 `ChatWriter` 实例的 `tools` 属性中：

In [ ]:
chat.tools

`tool_choice` 属性存储着工具选择模式，默认值为 `auto`：

In [ ]:
chat.tool_choice

## 调用

模型将在调用时自动选择工具，所有模式（流式/非流式、同步/异步）均支持。

In [ ]:
from langchain_core.messages import HumanMessage

messages = [
    HumanMessage(
        "Use knowledge graph tool to compose this answer. Tell me what th first line of documents stored in your KG. Also I want to know: how many SuperCopa trophies have Barcelona won?"
    )
]

response = chat.invoke(messages)
messages.append(response)

在函数工具的情况下，您将收到一个包含工具调用请求的助手消息。

In [ ]:
print(response.tool_calls)

然后，您可以手动处理工具调用请求，将其发送给模型并接收最终响应：

In [ ]:
for tool_call in response.tool_calls:
    selected_tool = {
        "get_supercopa_trophies_count": get_supercopa_trophies_count,
    }[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

response = chat.invoke(messages)
print(response.content)

通过 `GraphTool`，模型将远程调用它，并在 `additional_kwargs` 中 `graph_data` 键下返回使用信息：

In [ ]:
print(response.additional_kwargs["graph_data"])

`content` 属性包含最终响应：

In [ ]:
print(response.content)

## 链式调用

由于 Writer Graph 工具的特殊性（您无需手动调用它，Writer 服务器会自行调用并返回基于 RAG 的生成结果），无法单独调用它，因此 GraphTool 不能作为链的一部分使用。

## API 参考
有关 `GraphTool` 的所有功能和配置的详细文档，请访问 [API 参考](https://python.langchain.com/api_reference/writer/tools/langchain_writer.tools.GraphTool.html#langchain_writer.tools.GraphTool)。